In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import datetime


In [2]:
piramide_chile_INE = { # INE - Proyección base 2017
    '>=70': 1_614_364,
    '60-69': 1_857_879,
    '50-59': 2_392_614,
    '40-49': 2_658_453,
    '<=39': 11_155_053,
}
dosis2 = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto78/vacunados_edad_fecha_2daDosis_std.csv', parse_dates=[1],)
bins = [0, 40, 50, 60, 70, 999]
labels = ['<=39', '40-49', '50-59', '60-69', '>=70']
dosis2['agerange'] = pd.cut(dosis2.Edad, bins, labels=labels, include_lowest=True, right=False)
dosis2 = pd.pivot_table(dosis2, values=['Segunda Dosis'], index=['Fecha'],
                    columns=['agerange'], aggfunc=np.sum)['Segunda Dosis']
monodosis = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto78/vacunados_edad_fecha_UnicaDosis_std.csv', parse_dates=[1],)
monodosis['agerange'] = pd.cut(monodosis.Edad, bins, labels=labels, include_lowest=True, right=False)
monodosis = pd.pivot_table(monodosis, values=['Unica Dosis'], index=['Fecha'],
                    columns=['agerange'], aggfunc=np.sum)['Unica Dosis']
esquema_completo = dosis2.shift(14) + monodosis.shift(28)
esquema_completo = esquema_completo.cumsum()
p_esquema_completo = esquema_completo.copy()
p_esquema_completo['>=70'] = p_esquema_completo['>=70']/piramide_chile_INE['>=70']*100
p_esquema_completo['60-69'] = p_esquema_completo['60-69']/piramide_chile_INE['60-69']*100
p_esquema_completo['50-59'] = p_esquema_completo['50-59']/piramide_chile_INE['50-59']*100
p_esquema_completo['40-49'] = p_esquema_completo['40-49']/piramide_chile_INE['40-49']*100
p_esquema_completo['<=39'] = p_esquema_completo['<=39']/piramide_chile_INE['<=39']*100

etiquetas = {
    'value': 'Porcentaje de la población',
    'agerange': 'Rango etario'
}
pl_esq_com = px.line(p_esquema_completo, labels=etiquetas)
pl_esq_com.update_layout(title_text="Porcentaje de la población con esquema de vacunación completo (Producto 78)")
pl_esq_com.update_yaxes(range=[0,100],)
pl_esq_com.update_layout(hovermode='x')
pl_esq_com.update_traces(
    hovertemplate="<br>".join([
#         "Día: %{x}",
        "%{y:.1f}%",
    ])
)
pl_esq_com.update_layout(template='plotly_white')
# pl_vac_tot.update_layout(yaxis_tickformat = ',.1f')
pl_esq_com.update_layout(
    font=dict(
        size=14,
    )
)
pl_esq_com.add_layout_image(
    dict(
        source="https://i2.wp.com/dlab.cl/wp-content/uploads/2016/08/LogoWebDlab.png",
        xref="paper", yref="paper",
        x=1, y=1.00,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)
pl_esq_com.write_html('datacovidoutput/EsquemaCompleto.html')


In [3]:
casos = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto16/CasosGeneroEtario_std.csv')
tr_edad = {
    '00 - 04 años': '00-04',
    '05 - 09 años': '05-09',
    '10 - 14 años': '10-14',
    '15 - 19 años': '15-19',
    '20 - 24 años': '20-24',
    '25 - 29 años': '25-29',
    '30 - 34 años': '30-34',
    '35 - 39 años': '35-39',
    '40 - 44 años': '40-44',
    '45 - 49 años': '45-49',
    '50 - 54 años': '50-54',
    '55 - 59 años': '55-59',
    '60 - 64 años': '60-64',
    '65 - 69 años': '65-69',
    '70 - 74 años': '70-74',
    '75 - 79 años': '75-79',
    '80 y más años': '80+'
}
tr_sex ={
    'M': 'Hombre',
    'F': 'Mujer'
}
casos['Grupo de edad'].replace(tr_edad, inplace=True)
casos['Sexo'].replace(tr_sex, inplace=True)
casos['Fecha'] = pd.to_datetime(casos['Fecha'])
casos_genero_edad = pd.pivot_table(casos, values=['Casos confirmados'], index=['Fecha'],
                    columns=['Sexo', 'Grupo de edad'])['Casos confirmados']
# hay un dia que no hubieron nuevos casos, borrando
casos_genero_edad = casos_genero_edad.drop(pd.Timestamp('2020-10-02'))
casos_genero_edad = casos_genero_edad.resample('D').interpolate('quadratic')
casos_genero_edad = casos_genero_edad.diff().fillna(0).astype(int)
casos_edad = casos_genero_edad.sum(axis=1, level=1)
casos_edaduci = pd.DataFrame()
casos_edaduci['<=39'] = casos_edad[list(casos_edad.columns[:8])].sum(axis=1)
casos_edaduci['40-49'] = casos_edad[list(casos_edad.columns[8:10])].sum(axis=1)
casos_edaduci['50-59'] = casos_edad[list(casos_edad.columns[10:12])].sum(axis=1)
casos_edaduci['60-69'] = casos_edad[list(casos_edad.columns[12:14])].sum(axis=1)
casos_edaduci['>=70'] = casos_edad[list(casos_edad.columns[14:])].sum(axis=1)
casos_edaduci.sum(axis=1)

Fecha
2020-03-25       0
2020-03-26     240
2020-03-27     182
2020-03-28     289
2020-03-29     183
              ... 
2021-07-01    3501
2021-07-02    3665
2021-07-03    3823
2021-07-04    3989
2021-07-05    4150
Freq: D, Length: 468, dtype: int64

In [4]:
pl_casos = px.line(casos_edaduci.rolling(7).mean(), x=casos_edaduci.index, y=casos_edaduci.columns,
              labels={
                     "value": "Casos Diarios",
                     "x": "Día",
                     "variable": "Rango etario"
              },
              title="Media móvil 7d de los casos nuevos de COVID19 nacional (Producto 16)")

pl_casos.update_layout(hovermode='x')
pl_casos.update_layout(legend_traceorder='reversed')
pl_casos.update_traces(
    hovertemplate="<br>".join([
#         "Día: %{x}",
        "%{y:.0f}",
    ])
)
pl_casos.update_layout(template='plotly_white')
pl_casos.update_layout(yaxis_tickformat = ',.0f')
pl_casos.update_layout(
    font=dict(
        size=14,
    )
)
pl_casos.add_layout_image(
    dict(
        source="https://i2.wp.com/dlab.cl/wp-content/uploads/2016/08/LogoWebDlab.png",
        xref="paper", yref="paper",
        x=1, y=1.05,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)
pl_casos.add_vline(x=datetime.datetime(2021,4,15).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (>=70)',
#                        annotation_position="top left")
pl_casos.add_vline(x=datetime.datetime(2021,5,10).timestamp() * 1000, line_dash="dot",)
pl_casos.add_vline(x=datetime.datetime(2021,6,21).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (60-69)')
pl_casos.add_annotation(
    x=datetime.datetime(2021,4,15).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",

    text="80% Esquema completo (>=70)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-90,
    textangle=-90

)
pl_casos.add_annotation(
    x=datetime.datetime(2021,5,10).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (60-69)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
pl_casos.add_annotation(
    x=datetime.datetime(2021,6,21).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (50-59)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)

pl_casos.write_html('datacovidoutput/casos_nuevos_por_edad.html')

In [5]:
uci = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto9/HospitalizadosUCIEtario_T.csv')
uci['Grupo de edad'] = pd.to_datetime(uci['Grupo de edad'],infer_datetime_format=True)
uci = uci.rename(columns={'Grupo de edad': 'Fecha'}).set_index('Fecha')
uci

,<=39,40-49,50-59,60-69,>=70
Fecha,,,,,
2020-04-01,23,23,53,39,35
2020-04-02,23,27,67,46,37
2020-04-03,26,34,74,56,47
2020-04-04,40,36,84,68,52
2020-04-05,40,40,89,75,63
...,...,...,...,...,...
2021-07-06,504,432,579,698,488
2021-07-07,487,417,571,698,503
2021-07-08,455,398,558,680,470


In [6]:
pl_UCI = px.line(uci.rolling(7).mean(), x=uci.index, y=uci.columns,
              labels={
                     "value": "Ocupación UCI",
                     "x": "Día",
                     "variable": "Rango etario"
              },
              title="Media móvil 7d de la ocupación UCI de casos COVID19 nacional (Producto 9)")

pl_UCI.update_layout(hovermode='x')
pl_UCI.update_layout(legend_traceorder='reversed')
pl_UCI.update_traces(
    hovertemplate="<br>".join([
#         "Día: %{x}",
        "%{y:.0f}",
    ])
)
pl_UCI.update_layout(template='plotly_white')
pl_UCI.update_layout(yaxis_tickformat = ',.0f')
pl_UCI.update_layout(
    font=dict(
        size=14,
    )
)
pl_UCI.add_layout_image(
    dict(
        source="https://i2.wp.com/dlab.cl/wp-content/uploads/2016/08/LogoWebDlab.png",
        xref="paper", yref="paper",
        x=1, y=1.05,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)

pl_UCI.add_vline(x=datetime.datetime(2021,4,15).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (>=70)',
#                        annotation_position="top left")
pl_UCI.add_vline(x=datetime.datetime(2021,5,10).timestamp() * 1000, line_dash="dot",)
pl_UCI.add_vline(x=datetime.datetime(2021,6,21).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (60-69)')
pl_UCI.add_annotation(
    x=datetime.datetime(2021,4,15).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",

    text="80% Esquema completo (>=70)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-150,
    ay=-90,
    textangle=-90

)
pl_UCI.add_annotation(
    x=datetime.datetime(2021,5,10).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (60-69)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
pl_UCI.add_annotation(
    x=datetime.datetime(2021,6,21).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (50-59)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
pl_UCI.write_html('datacovidoutput/ocupacion_UCI_por_edad.html')



In [7]:
import pandas as pd
import numpy as np
import urllib.request
from zipfile import ZipFile
from re import compile
from pathlib import Path
from shutil import rmtree
deis_data = Path('deis_data')
deis_data.mkdir(parents=True, exist_ok=True)

def get_deis_death_url():
    datapattern = compile('http.*DEFUNCIONES_FUENTE_DEIS_2016_2021.*zip')
    with urllib.request.urlopen('https://deis.minsal.cl/wp-admin/admin-ajax.php?action=wp_ajax_ninja_tables_public_action&table_id=2889&target_action=get-all-data&default_sorting=manual_sort') as f:
        return datapattern.search(f.read().decode().replace(',','\n')).group().replace('\\', '')

def get_csv_deis():
    url = get_deis_death_url()
    urllib.request.urlretrieve(url, 'deis_data/' + url.split('/')[-1])
    with ZipFile('deis_data/' + url.split('/')[-1], 'r') as zip_ref:
        zip_ref.extractall('deis_data')
    return url.split('/')[-1][:-3]
def annos(row):
    edad = row['EDAD_CANT']
    tipo = row['EDAD_TIPO']
    if tipo == 1:
        return edad
    elif tipo == 2:
        return edad/12
    elif tipo == 3:
        return edad/365
    elif tipo == 4:
        return edad/365*24
    else:
        return np.nan
deis_csv = 'deis_data/' + get_csv_deis() + 'csv'

columnas = ['ANO_DEF', 'FECHA_DEF', 'GLOSA_SEXO', 'EDAD_TIPO', 'EDAD_CANT', 'CODIGO_COMUNA_RESIDENCIA', 'GLOSA_COMUNA_RESIDENCIA', 'GLOSA_REG_RES', 'DIAG1', 'CAPITULO_DIAG1', 'GLOSA_CAPITULO_DIAG1', 'CODIGO_GRUPO_DIAG1', 'GLOSA_GRUPO_DIAG1', 'CODIGO_CATEGORIA_DIAG1', 'GLOSA_CATEGORIA_DIAG1', 'CODIGO_SUBCATEGORIA_DIAG1', 'GLOSA_SUBCATEGORIA_DIAG1', 'DIAG2', 'CAPITULO_DIAG2', 'GLOSA_CAPITULO_DIAG2', 'CODIGO_GRUPO_DIAG2', 'GLOSA_GRUPO_DIAG2', 'CODIGO_CATEGORIA_DIAG2', 'GLOSA_CATEGORIA_DIAG2', 'CODIGO_SUBCATEGORIA_DIAG2', 'GLOSA_SUBCATEGORIA_DIAG2']
deis = pd.read_csv(deis_csv, sep=';', parse_dates=[1], header=None, names=columnas, index_col=False)
deis.set_index(['FECHA_DEF'], inplace=True)
deis.sort_index(inplace=True)
# CODIGO_CATEGORIA_DIAG1 U07 > covid19
rmtree(deis_data)
deis['EDAD_ANOS'] = deis.apply(annos, axis = 1)
bins = [0, 40, 50, 60, 70, 999]
labels = ['<=39', '40-49', '50-59', '60-69', '>=70']
deis['agerange'] = pd.cut(deis.EDAD_ANOS, bins, labels=labels, include_lowest=True, right=False)
defunciones_deis_genero_edad = pd.pivot_table(deis.loc[(deis['CODIGO_CATEGORIA_DIAG1'] == 'U07')], values=['EDAD_CANT'], index=['FECHA_DEF'],
                    columns=['GLOSA_SEXO', 'agerange'], aggfunc='count')['EDAD_CANT']
defunciones_deis_genero_edad.columns.names = ['Sexo', 'Edad']
defunciones_deis_genero_edad.index.name = 'Fecha'
defunciones_deis_genero_edad = defunciones_deis_genero_edad.asfreq('D').fillna(0)
defunciones_deis_genero_edad


Sexo       Hombre                         Mujer                        
Edad         <=39 40-49 50-59 60-69  >=70  <=39 40-49 50-59 60-69  >=70
Fecha                                                                  
2020-03-16    0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0
2020-03-17    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
2020-03-18    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0
2020-03-19    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0
2020-03-20    0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0
...           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...
2021-07-03    2.0   5.0   6.0  11.0  36.0   1.0   3.0   6.0  10.0  51.0
2021-07-04    3.0   3.0  10.0  14.0  43.0   1.0   2.0  12.0  11.0  27.0
2021-07-05    6.0   4.0   9.0  17.0  41.0   1.0   4.0   7.0   5.0  32.0
2021-07-06    4.0   2.0   6.0  12.0  39.0   2.0   4.0   1.0   8.0  34.0
2021-07-07    1.0   1.0   0.0   1.0  12.0   0.0   1.0   1.0   1.0   4.0

[479 rows x 10 columns]

In [8]:
defunciones_deis_edad = defunciones_deis_genero_edad.sum(axis=1, level=1)
defunciones_deis_edad

Edad,<=39,40-49,50-59,60-69,>=70
Fecha,,,,,
2020-03-16,0.0,0.0,0.0,0.0,1.0
2020-03-17,0.0,0.0,0.0,0.0,0.0
2020-03-18,0.0,0.0,0.0,1.0,0.0
2020-03-19,0.0,0.0,0.0,1.0,0.0
2020-03-20,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...
2021-07-03,3.0,8.0,12.0,21.0,87.0
2021-07-04,4.0,5.0,22.0,25.0,70.0
2021-07-05,7.0,8.0,16.0,22.0,73.0


In [9]:
pl_deis = px.line(defunciones_deis_edad.rolling(7).mean(), x=defunciones_deis_edad.index, y=defunciones_deis_edad.columns,
              labels={
                     "value": "Defunciones",
                     "x": "Día",
                     "variable": "Rango etario"
              },
              title="Media móvil 7d de las defunciones por COVID19 nacional (DEIS)")

pl_deis.update_layout(hovermode='x')
pl_deis.update_layout(legend_traceorder='reversed')
pl_deis.update_traces(
    hovertemplate="<br>".join([
#         "Día: %{x}",
        "%{y:.0f}",
    ])
)
pl_deis.update_layout(template='plotly_white')
pl_deis.update_layout(yaxis_tickformat = ',.0f')
pl_deis.update_layout(
    font=dict(
        size=14,
    )
)
pl_deis.add_layout_image(
    dict(
        source="https://i2.wp.com/dlab.cl/wp-content/uploads/2016/08/LogoWebDlab.png",
        xref="paper", yref="paper",
        x=1, y=1.05,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)

pl_deis.add_vline(x=datetime.datetime(2021,4,15).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (>=70)',
#                        annotation_position="top left")
pl_deis.add_vline(x=datetime.datetime(2021,5,10).timestamp() * 1000, line_dash="dot",)
pl_deis.add_vline(x=datetime.datetime(2021,6,21).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (60-69)')
pl_deis.add_annotation(
    x=datetime.datetime(2021,4,15).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",

    text="80% Esquema completo (>=70)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-150,
    ay=-90,
    textangle=-90

)
pl_deis.add_annotation(
    x=datetime.datetime(2021,5,10).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (60-69)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
pl_deis.add_annotation(
    x=datetime.datetime(2021,6,21).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (50-59)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
pl_deis.write_html('datacovidoutput/defunciones_por_edad.html')


In [10]:
cfr_semanal = px.line((defunciones_deis_edad.resample('W').sum().shift(2) / casos_edaduci.resample('W').sum()).rolling(4).mean(),
                      labels={
                     "value": "Tasa de letalidad",
                     "x": "Día",
                     "variable": "Rango etario"
                      },
                      title="Media móvil mensual de la tasa de letalidad semanal por COVID19")
cfr_semanal.update_layout(hovermode='x')
cfr_semanal.update_layout(legend_traceorder='reversed')
cfr_semanal.update_traces(
    hovertemplate="<br>".join([
#         "Día: %{x}",
        "%{y:.2%}",
    ])
)
cfr_semanal.update_layout(template='plotly_white')
cfr_semanal.update_layout(yaxis_tickformat = '.1%')
cfr_semanal.update_layout(
    font=dict(
        size=14,
    )
)
cfr_semanal.add_layout_image(
    dict(
        source="https://i2.wp.com/dlab.cl/wp-content/uploads/2016/08/LogoWebDlab.png",
        xref="paper", yref="paper",
        x=1, y=1.05,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)

cfr_semanal.add_vline(x=datetime.datetime(2021,4,15).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (>=70)',
#                        annotation_position="top left")
cfr_semanal.add_vline(x=datetime.datetime(2021,5,10).timestamp() * 1000, line_dash="dot",)
cfr_semanal.add_vline(x=datetime.datetime(2021,6,21).timestamp() * 1000, line_dash="dot",)
#                         annotation_text='Vacunacion del 80% (60-69)')
cfr_semanal.add_annotation(
    x=datetime.datetime(2021,4,15).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",

    text="80% Esquema completo (>=70)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-150,
    ay=-90,
    textangle=-90

)
cfr_semanal.add_annotation(
    x=datetime.datetime(2021,5,10).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (60-69)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
cfr_semanal.add_annotation(
    x=datetime.datetime(2021,6,21).timestamp() * 1000,
    y=0.99,
    xref="x",
    yref="y domain",
    text="80% Esquema completo (50-59)",
    showarrow=False,
    font=dict(
        size=16,
        ),
    xshift=-10,
    arrowhead=1,
    ax=-190,
    ay=-150,
    textangle=-90
)
cfr_semanal.write_html('datacovidoutput/CFR_semanal.html')


In [11]:
defunciones_deis_edad.resample('2W').sum()

Edad,<=39,40-49,50-59,60-69,>=70
Fecha,,,,,
2020-03-22,0.0,0.0,0.0,3.0,6.0
2020-04-05,2.0,3.0,6.0,14.0,57.0
2020-04-19,2.0,4.0,22.0,27.0,131.0
2020-05-03,7.0,14.0,20.0,41.0,180.0
2020-05-17,21.0,36.0,66.0,125.0,417.0
2020-05-31,70.0,75.0,213.0,384.0,1324.0
2020-06-14,77.0,115.0,306.0,667.0,2158.0
2020-06-28,64.0,86.0,306.0,638.0,1758.0
2020-07-12,46.0,79.0,224.0,434.0,1221.0
